# two_party_hybrid_input_binary_classification 

## 資料品質檢查

### 設定資料路徑 & 參數

In [1]:
import os
guest, host = 9999, 10000
data_base = "/data/projects/fate/"

dense_data = {"name": "titanic_hetero_guest", "namespace": f"experiment"}
dense_data_dir = os.path.join(data_base, "persistence/data/titanic_hetero_guest.csv")

### 缺失值 & 欄位名

#### 欄位名請全部調整成小寫
#### Age欄位有缺失值, 等等可以使用 DataTransform 來填平均年齡 29.7

In [2]:
import pandas as pd
dense_df = pd.read_csv(dense_data_dir)
print(dense_df.isna().sum())
print(dense_df.head(5))
print(f"The Average Age of Passenger is {dense_df.age.mean():.2f}")

passengerid      0
survived         0
pclass           0
sex              0
age            177
sibsp            0
dtype: int64
   passengerid  survived  pclass     sex   age  sibsp
0            1         0       3    male  22.0      1
1            2         1       1  female  38.0      1
2            3         1       3  female  26.0      0
3            4         1       1  female  35.0      1
4            5         0       3    male  35.0      0
The Average Age of Passenger is 29.70


## 上傳資料

In [3]:
from pipeline.backend.pipeline import PipeLine
pipeline_upload = PipeLine().set_initiator(role='guest', party_id=guest).set_roles(guest=guest)
partition = 4

pipeline_upload.add_upload_data(file=dense_data_dir,
                                table_name=dense_data["name"],             # table name
                                namespace=dense_data["namespace"],         # namespace
                                head=1, partition=partition)               # data info
pipeline_upload.upload(drop=1)

 UPLOADING:||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||100.00%

2023-06-06 06:37:45.655 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060637455117550

2023-06-06 06:37:45.662 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00


m2023-06-06 06:37:47.687 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 06:37:47.688 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:02
2023-06-06 06:37:48.704 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:03
2023-06-06 06:37:49.722 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:04
2023-06-06 06:37:50.736 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:05
2023-06-06 06:37:51.760 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:06
2023-06-06 06:37:52.775 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component upload_0, time elapse: 0:00:07
2023-06-06 

### 建構 Training pipeline 範例

使用 `Reader` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 Reader`

In [4]:
from pipeline.component import Reader
reader_0 = Reader(name="reader_0")

# set guest parameter
reader_0.get_party_instance(role='guest', party_id=guest).component_param(
    table={"name": "titanic_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=host).component_param(
    table={"name": "titanic_hetero_host", "namespace": "experiment"})

使用 `DataTransform` 模塊來讀取資料, 注意這邊如果有 `K 個 Party` 參與則有 `K 個 DataTransform`

`DataTransform` 負責資料前處理( 設定目標欄位名稱, 補缺值, 替換 outliers )

同時也可以設定各欄位的資料格式, `float`, `str`, `int` 等等

In [5]:
from pipeline.component import DataTransform
data_transform_0 = DataTransform(name="data_transform_0")

# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=guest).component_param(
    with_label=True, label_name="survived", label_type="int",
    missing_fill=True, missing_fill_method="designated", default_value=["O", "male", 29.7, 0], outlier_replace=False,
    data_type="float", exclusive_data_type={"pclass":"str", "sex":"str", "sibsp":"str"}
)

data_transform_0.get_party_instance(role='host', party_id=[host]).component_param(
    with_label=False,
    missing_fill=True, missing_fill_method="designated", default_value=["100", "0.0", "N"], outlier_replace=False,
    data_type="float", exclusive_data_type={"parch":"str", "embarked":"str"}
)

In [6]:
from pipeline.component import OneHotEncoder

onehot_encoder_0 = OneHotEncoder(name="onehot_encoder_0")

onehot_encoder_0.get_party_instance(role='guest', party_id=guest).component_param(
    transform_col_indexes=[0, 1, 3], transform_col_names=["pclass", "sex", "sibsp"]
)

onehot_encoder_0.get_party_instance(role='host', party_id=[host]).component_param(
    transform_col_indexes=[0, 2], transform_col_names=["parch", "embarked"]
)

使用 `Intersection` 模塊來達到對齊雙方相同顧客 ID, 且不會洩漏非相同顧客的 ID

In [7]:
from pipeline.component import Intersection
intersect_0 = Intersection(name="intersect_0")

現在使用 `HeteroSecureBoost` 模塊. 用以下的參數來構建樹模型

In [8]:
from pipeline.component import HeteroSecureBoost
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


最後, 為了檢驗好壞 使用 `Evaluation` 模塊來驗證好壞

In [9]:
from pipeline.component import Evaluation
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

使用 `pipeline` 模塊來構建聯邦學習流程
實例化 `pipeline` 並設定 `initiator` 和 `roles`:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
        
上一個 component 的 output 是下一個 component 的 input

    - data_transform_0 吃 reader_0 的 output
    - intersect_0 吃 data_transform_0 的 output
    - hetero_secureboost_0 吃 intersect_0 的 output
    - evaluation_0 吃 hetero_secureboost_0 的 output (預測值)

記得用 `pipeline.compile()` 來打包整串流程
使用 `pipeline.fit()` 來開始進行訓練

In [ ]:
from pipeline.backend.pipeline import PipeLine
from pipeline.interface import Data
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=guest) \
        .set_roles(guest=guest, host=host)

pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(onehot_encoder_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=onehot_encoder_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile()
pipeline.fit()

2023-06-06 06:37:57.266 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202306060637569089540

2023-06-06 06:37:57.274 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2023-06-06 06:37:58.295 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2023-06-06 06:37:58.296 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:01
2023-06-06 06:37:59.311 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2023-06-06 06:38:00.328 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2023-06-06 06:38:01.385 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2023-06-06 06:38:02.402 | INFO     

2023-06-06 06:38:35.926 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:38
2023-06-06 06:38:37.085 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:39
2023-06-06 06:38:38.104 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:40
2023-06-06 06:38:39.127 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:41
2023-06-06 06:38:40.143 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:42
2023-06-06 06:38:41.215 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component data_transform_0, time elapse: 0:00:43
2023-06-06 06:38:42.335 | INFO     | pipeline.utils.invoker.job_

2023-06-06 06:39:15.913 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:18
2023-06-06 06:39:16.932 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:19
2023-06-06 06:39:18.015 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:20
2023-06-06 06:39:19.099 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component intersect_0, time elapse: 0:01:21


當訓練結束後, 模型會用來做預測. 使用者可以自由選擇要不要儲存此次 `pipeline` 以方便未來重複使用
使用 `pipeline.dump(pipeline_saved_path)` 來完成儲存

In [ ]:
pipeline.dump("pipeline_saved/pipeline_saved_hybrid.pkl");

### 建構 Inference pipeline 範例

首先, 使用 `PipeLine.load_model_from_file` load `pkl` 檔

部署 Inference 需要的模塊, 在這邊是 `data_transform_0`, `onehot_encoder_0`, `intersect_0`, `hetero_secureboost_0`

In [ ]:
pipeline = PipeLine.load_model_from_file('pipeline_saved/pipeline_saved_hybrid.pkl')
pipeline.deploy_component([pipeline.data_transform_0, 
                           pipeline.onehot_encoder_0, 
                           pipeline.intersect_0, 
                           pipeline.hetero_secureboost_0]);

接著, 部署 `Reader` 模塊 `reader_1` 來讀取新data

In [ ]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=guest).component_param(table={"name": "titanic_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=host).component_param(table={"name": "titanic_hetero_host", "namespace": "experiment"})

最後, 部署新的 `Evaluation` 來衡量 predict ( Inference ) 的表現

In [ ]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

整合所有模塊

In [ ]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


預測!

In [ ]:
predict_pipeline.predict()

用 `pipeline.get_component('evaluation_0').get_summary()` 

來取得 `evaluation_0` 模塊的資訓儲並存成 json 檔

In [ ]:
import json
data_base = "/data/projects/fate/"
metadata_saved_dir = os.path.join(data_base, "persistence/metadata/two_party_hybrid_input_binary_classification.json")
metedata = json.dumps(pipeline.get_component('evaluation_0').get_summary(), indent=4)

with open(metadata_saved_dir, "w") as json_file:
    json_file.write(metedata)
                                  
print(f"Write in metadata_saved_dir : {metadata_saved_dir} \n {metedata}")